# AKS MCP Demo - Working Version

Simple, working AKS MCP integration based on your test file.

In [66]:
import asyncio
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
print("✅ Setup complete")

✅ Setup complete


In [67]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory

print("✅ Imports successful")

✅ Imports successful


In [68]:
# Initialize kernel
kernel = Kernel()

# Configure Azure OpenAI
azure_openai = AzureChatCompletion(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
)

kernel.add_service(azure_openai)
print("✅ Kernel initialized")

✅ Kernel initialized


In [69]:
# Setup AKS MCP Plugin
aks_mcp_path = "../../aks-mcp"
if not Path(aks_mcp_path).exists():
    print(f"❌ AKS-MCP not found at {aks_mcp_path}")
    raise FileNotFoundError("Build with: make build")

mcp_plugin = MCPStdioPlugin(
    name="AKSMCP",
    command=aks_mcp_path,
    args=["--transport", "stdio", "--access-level", "admin"],
)

await mcp_plugin.connect()
kernel.add_plugin(mcp_plugin, plugin_name="akstool")
print("✅ MCP connected")

✅ MCP connected


In [70]:
# Show available functions
plugin_functions = kernel.get_plugin("akstool")
print(f"📋 Available functions: {len(plugin_functions.functions)}")
for func_name in list(plugin_functions.functions.keys())[:5]:
    print(f"  • {func_name}")

📋 Available functions: 16
  • az_advisor_recommendation
  • az_aks_operations
  • az_fleet
  • az_monitoring
  • az_network_resources


In [71]:
# Create agent
agent = ChatCompletionAgent(
    kernel=kernel,
    name="AKS_Assistant",
    instructions="""You are an expert AKS administrator with access to powerful AKS management tools.

IMPORTANT: Always use the available akstool functions to gather information instead of asking users for details.

When asked about clusters, detectors, or recommendations:
1. FIRST use az_aks_operations to discover available clusters and their details
2. THEN use the appropriate tool with the discovered information
3. Provide comprehensive analysis based on the actual data

Never ask users for cluster names, resource groups, or subscription IDs - discover them using the tools."""
)

print("✅ Agent created")

✅ Agent created


In [72]:
# Simple helper function
async def ask_assistant(question: str):
    print(f"🙋 Question: {question}")
    print("-" * 50)
    
    chat_history = ChatHistory()
    chat_history.add_user_message(question)
    
    async for response in agent.invoke(messages=chat_history):
        if response.content:
            print(response.content, end="", flush=True)
    
    print("\n" + "=" * 50)

print("✅ Helper function ready")

✅ Helper function ready


In [73]:
# Test: Cluster discovery
await ask_assistant("What AKS clusters do I have? Please provide a comprehensive overview.")

🙋 Question: What AKS clusters do I have? Please provide a comprehensive overview.
--------------------------------------------------
Here is a comprehensive overview of your Azure Kubernetes Service (AKS) clusters:

### Cluster 1: Hub
- **Name:** hub
- **Resource Group:** FL_hieunhu_testhubfleet1_eastasia
- **Location:** eastasia
- **Kubernetes Version:** 1.33.2
- **Provisioning State:** Succeeded
- **AAD Profile:**
  - Enabled Azure RBAC
  - Managed AAD
- **API Server Access:**
  - Disable Run Command: True
- **Upgrade Profile:** 
  - Node OS Upgrade Channel: NodeImage
  - Upgrade Channel: patch
- **RBAC Enabled:** True
- **DNS Prefix:** testhubfleet1-dns
- **FQDN:** testhubfleet1-dns-dm34i8zt.hcp.eastasia.azmk8s.io
- **Agent Pool:**
  - Name: agentpool
  - Count: 1
  - VM Size: Standard_D4s_v4
  - OS Disk Size: 128 GB
  - OS Type: Linux
  - Current Orchestrator Version: 1.33.2
  - Network Profile: None
  - Power State: Running
- **Network Profile:**
  - Plugin: azure
  - Load Balance

In [74]:
# Test: Diagnostic detectors
await ask_assistant("What diagnostic detectors are available? Discover my clusters first, then list detectors.")

🙋 Question: What diagnostic detectors are available? Discover my clusters first, then list detectors.
--------------------------------------------------
### Available Diagnostic Detectors

#### Cluster: `hub`
- **Active Live Site Incidents:** Single source of reference for any active live site incidents.  
  Category: Cluster and Control Plane Availability and Performance
- **Pod CIDR out of IP Ranges:** Checks for cluster nodes that were not able to be assigned an IP range due to the pod CIDR running out of IP ranges.  
  Category: Node Health
- **AKS Operation Troubleshooter:** Checks for a failing operation ID then displays the summary and customer mitigation for that error type.  
  Category: Workflow Nodes
- **AKS-managed AAD Integration Issues:** Detects any issues with AKS-managed AAD integration for operator access.  
  Category: Identity and Security
- **API Server - Resource Intensive Listing:** Detect potential API Server load issues that stem from large and or inefficient l

In [75]:
# Test: Workload analysis
await ask_assistant("Analyze the workloads in my clusters. Find my clusters first, then check pods and services.")

🙋 Question: Analyze the workloads in my clusters. Find my clusters first, then check pods and services.
--------------------------------------------------
It appears that there are permission issues with accessing the credentials for your clusters, which is preventing further analysis of the workloads.

Here is what I have tried so far:
1. Listed the available clusters - successfully identified two clusters: `hub` and `akskeda-aks`.
2. Attempted to retrieve pods and services details from both clusters using different methods, but encountered errors including permission issues when fetching the credentials.

To successfully analyze workloads in your clusters, please ensure that the client account has the necessary permissions for accessing AKS cluster resources, particularly for listing cluster user credentials and running kubectl commands.

If you need further assistance or have specific actions that should be directed differently, let me know!


In [76]:
# Custom question
await ask_assistant("Tell me about the network configuration and suggest improvements.")

🙋 Question: Tell me about the network configuration and suggest improvements.
--------------------------------------------------
Based on the network configurations of the two AKS clusters ("hub" in "FL_hieunhu_testhubfleet1_eastasia" and "akskeda-aks" in "akskeda-rg"), here is the detailed information and some recommendations for improvement:

### Network Configuration Details:

#### Cluster: "hub"

1. **Load Balancer**:
   - **Backend Address Pools**: Utilizes two backend pools for outbound and cluster services.
   - **Frontend IP Configurations**: Configured with a dynamic IP allocation and a standard public IP.
   - **Outbound Rules**: Configured with an outbound rule enabling outbound traffic with TCP reset and specific idle timeout.

2. **Network Security Group (NSG)**:
   - **Default Security Rules**: Includes standard rules allowing VNet traffic, Azure Load Balancer traffic, and denying all inbound/outbound traffic except for Internet outbound.
   - **Custom Rules**: No additio

In [77]:
# Cleanup
try:
    await mcp_plugin.close()
    print("✅ MCP plugin disconnected")
except Exception as e:
    print(f"⚠️ Cleanup warning: {e}")
    
print("✅ Demo complete")

⚠️ Cleanup warning: Attempted to exit cancel scope in a different task than it was entered in
✅ Demo complete
